Build a CNN model for Bird species

Bird species classification is the process of using machine learning and computer vision techniques to identify and categorize different species of birds based on their visual characteristics. By analyzing images of birds, models can extract features and patterns to accurately classify bird species. This classification is vital for ecological research, wildlife monitoring, and conservation efforts. Advancements in deep learning and the availability of large annotated datasets have improved the accuracy of bird species classification models. Challenges include variations in lighting, pose, and background clutter. Ongoing research focuses on methods like transfer learning and data augmentation to enhance classification performance and contribute to avian biodiversity understanding and conservation.

In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d akash2907/bird-species-classification
!unzip bird-species-classification.zip

Saving kaggle.json to kaggle.json
100% 1.37G/1.37G [01:01<00:00, 24.5MB/s]
100% 1.37G/1.37G [01:01<00:00, 24.1MB/s]
Archive:  bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test

In [ ]:
import numpy as np
import pandas as pd
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential

import numpy as np
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
root_train = '/content/train_data/train_data'
root_test = '/content/test_data/test_data'

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(120, 120),
                                      class_mode='categorical',
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical',
                                      batch_size=8)

train.class_indices

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [ ]:
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 12)     48        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 24)       9

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [ ]:
model.fit(train,batch_size=8,validation_data=test,epochs=50,callbacks=early_stop)
# model.fit(train,batch_size=8,validation_data=test,epochs=50)

Epoch 1/50
19/19 [==============================] - 95s 5s/step - loss: 2.8204 - accuracy: 0.0600 - val_loss: 2.7530 - val_accuracy: 0.1274
Epoch 2/50
19/19 [==============================] - 88s 5s/step - loss: 2.5444 - accuracy: 0.1533 - val_loss: 2.7899 - val_accuracy: 0.1274
Epoch 3/50
19/19 [==============================] - 86s 5s/step - loss: 2.4427 - accuracy: 0.2133 - val_loss: 2.8240 - val_accuracy: 0.0318
Epoch 4/50
19/19 [==============================] - 118s 6s/step - loss: 2.3917 - accuracy: 0.2000 - val_loss: 2.8868 - val_accuracy: 0.0637
Epoch 5/50
19/19 [==============================] - 87s 5s/step - loss: 2.3230 - accuracy: 0.2333 - val_loss: 2.8661 - val_accuracy: 0.1083
Epoch 6/50
19/19 [==============================] - 87s 5s/step - loss: 2.1905 - accuracy: 0.3333 - val_loss: 2.8880 - val_accuracy: 0.0701
Epoch 7/50
19/19 [==============================] - 85s 5s/step - loss: 2.1972 - accuracy: 0.2733 - val_loss: 2.9667 - val_accuracy: 0.1210
Epoch 8/50
19/19 [=

In [ ]:
model.save('birds.h5')

In [ ]:
img1 = image.load_img('/content/test_data/test_data/bonegl/DSC_4587.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti', 'bonegl', "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul",
          "himgri", "hsparo", "indvul", "jglowl", "lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 20ms/step
12
lbicrw


In [ ]:
model.predict(test)

20/20 [==============================] - 49s 2s/step


array([[1.1233986e-07, 4.9376704e-06, 1.8252984e-02, ..., 3.4602161e-03,
        1.0264011e-04, 2.6410059e-03],
       [2.0909190e-02, 6.1845931e-05, 4.2298593e-02, ..., 1.1130028e-03,
        3.4146165e-04, 1.1502585e-02],
       [6.2152135e-06, 4.8486558e-03, 6.0321355e-01, ..., 6.3185074e-04,
        4.7989834e-05, 1.4349729e-05],
       ...,
       [8.4027704e-03, 1.2670977e-01, 1.5563621e-01, ..., 4.5858221e-03,
        5.3469092e-04, 5.4525392e-04],
       [1.6065569e-03, 4.3958877e-04, 2.0196366e-03, ..., 3.1656607e-03,
        3.9281170e-03, 1.6942264e-01],
       [2.4716737e-04, 5.5666394e-03, 2.1551005e-03, ..., 1.7108854e-02,
        1.4055556e-02, 7.5529129e-03]], dtype=float32)

In [ ]:
test_loss, test_accuracy = model.evaluate(test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

20/20 [==============================] - 48s 2s/step - loss: 3.9730 - accuracy: 0.2357
Test Loss: 3.973012685775757
Test Accuracy: 0.23566879332065582
